In [2]:
from pyiron_atomistics import Project
import os
import numpy as np
import matplotlib.pyplot as plt
from helpers import *

In [ ]:
from pychromatic import Multiplot
from pychromatic.colors import accent

In [3]:
pr = Project("mp")

This is the input used for initial calculations, probably needs to be increased

In [ ]:
n_eq = 20000
n_int = 50000
n_iter = 1
N = 8

Composition and temp array, for testing, probably better to use very few values

In [ ]:
comp_array = np.arange(0, 0.6, 0.1)
temp_array = np.arange(500, 1000, 50)

In [ ]:
for comp in comp_array:
    for temp in temp_array:
        #fcc job
        job = pr.create.job.Calphy('fcc_%.2f_%d'%(comp, int(temp)), 
                                   delete_aborted_job=True, 
                                   delete_existing_job=True)
        structure = pr.create.structure.bulk("Al", cubic=True).repeat(N)
        job.structure = replace_atom(structure, 'Li', to_replace=int(N**3*4*comp))
        job.potential = 'AlLi-atomicrex'
        job.server.cores = 16
        #job.server.queue = "s_cmfe"
        job.calc_free_energy(temperature=int(temp), 
                             pressure=0,
                             n_equilibration_steps=n_eq, 
                             n_switching_steps=n_int,
                             n_iterations=n_iter,
                             reference_phase="solid")
        job.run()

        #lqd job
        job = pr.create.job.Calphy('lqd_%.2f_%d'%(comp, int(temp)), 
                                   delete_aborted_job=True, 
                                   delete_existing_job=True)
        structure = pr.create.structure.bulk("Al", cubic=True).repeat(N)
        job.structure = replace_atom(structure, 'Li', to_replace=int(N**3*4*comp))
        job.potential = 'AlLi-atomicrex'
        job.server.cores = 16
        #job.server.queue = "s_cmfe"
        job.calc_free_energy(temperature=int(temp), 
                             pressure=0,
                             n_equilibration_steps=n_eq, 
                             n_switching_steps=n_int,
                             n_iterations=n_iter,
                             reference_phase="liquid")
        job.run()
        
        #b32 job
        job = pr.create.job.Calphy('b32_%.2f_%d'%(comp, int(temp)), 
                                   delete_aborted_job=True, 
                                   delete_existing_job=True)
        structure = pr.create.structure.ase.read('LiAl_pos2', format='vasp')
        job.structure = replace_atoms(structure, 'Al', int(len(structure)/2-len(structure)*comp))
        job.potential = 'AlLi-atomicrex'
        job.server.cores = 16
        #job.server.queue = "s_cmfe"
        job.calc_free_energy(temperature=int(temp), 
                             pressure=0,
                             n_equilibration_steps=n_eq, 
                             n_switching_steps=n_int,
                             n_iterations=n_iter,
                             reference_phase="solid")
        job.run()

Analysis

In [ ]:
df = create_dataframe(pr)

The analysis is at the moment done for different temperature values manually. The reason is so that to ensure common tangent construction makes sense. We could do this automatically..

In [ ]:
phases = []

In [ ]:
temp = 900
fitorder = 2
conc, fcc, fit_fcc = extract_fe(df, 'fcc', temp, fitorder=fitorder)
conc, b32, fit_b32 = extract_fe(df, 'b32', temp, fitorder=fitorder)
conc, lqd, fit_lqd = extract_fe(df, 'lqd', temp, fitorder=fitorder)
norms  = normalise_curves([lqd, fcc, b32], conc)

In [ ]:
cx1, cf1, cost1 = find_common_tangent(conc, norms[0], norms[1])
cx2, cf2, cost2 = find_common_tangent(conc, norms[0], norms[2])
cx3, cf3, cost3 = find_common_tangent(conc, norms[1], norms[2])
print(cx1, cf1, cost1)
print(cx2, cf2, cost2)
print(cx3, cf3, cost3)

In [ ]:
mlt = Multiplot(width=300)
mlt[0,0].plot(conc, norms[0], label='lqd', color=accent['pblue'])
mlt[0,0].plot(conc, norms[1], label='fcc', color=accent['pred'])
mlt[0,0].plot(conc, norms[2], label='b32', color=accent['pgreen'])
mlt[0,0].legend(frameon=False)
mlt[0,0].set_xlabel(r'$x_\mathrm{Li}$ (at. w%)', fontsize=10)
mlt[0,0].set_ylabel('F (eV)', fontsize=10)
mlt[0,0].set_title(f'T={temp} K', fontsize=10);

mlt[0,0].plot(cx1, cf1, color=accent['lgrey'], ls='dashed')
mlt[0,0].plot(cx2, cf2, color=accent['lgrey'], ls='dotted')
mlt[0,0].plot(cx3, cf3, color=accent['lgrey'], ls='dotted')

Only relevant values are added to the `phases` list

In [ ]:
phases.append({'conc': cx1, 't':[temp, temp], 'c': accent['pred']})
phases.append({'conc': cx2, 't':[temp, temp], 'c': accent['pgreen']})
#phases.append({'conc': cx3, 't':[temp, temp], 'c': accent['pyellow']})

Repeat for other temperatures,

Not a great plot, but gives the salient points:

In [ ]:
for p in phases:
    plt.plot(p['conc'], p['t'], color=p['c'])